<div style="background-color: #003366; color: #fff; padding: 10px; border-radius: 5px; text-align: center;">
    <h3>Brain Tumor Analysis</h3>
</div>
<div style="background-color: #000; border-radius: 2px; border: 1px solid #000; padding: 15px; font-size: 100%; text-align: center;">
    <img src="https://media.giphy.com/media/du3J3cXyzhj75IOgvA/giphy.gif" alt="GitHub GIF" style="max-width: 100%; height: auto;">
</div>


In [ ]:
!pip install -q git+https://github.com/THU-MIG/yolov10.git

In [ ]:
!wget -P -q https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10n.pt

In [ ]:
!pip install -q roboflow

<div style="background-color: #003366; color: #fff; padding: 10px; border-radius: 5px; text-align: center;">
    <h3>DATA SET</h3>
</div>

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="REPLACE WITH ROBOFLOW API KEY FROM UNIVERSE")
project = rf.workspace("brain-mri").project("mri-rskcu")
version = project.version(3)
dataset = version.download("yolov9")

<div style="background-color: #003366; color: #fff; padding: 10px; border-radius: 5px; text-align: center;">
    <h3>TRANING MODEL </h3>
</div>

In [ ]:
!yolo task=detect mode=train epochs = 25, batch=32 plots=True \
model = '/content/-q/yolov10n.pt' \
data = '/content/MRI-3/data.yaml'

In [ ]:
from ultralytics import YOLOv10

model_path = "/content/runs/detect/train9/weights/best.pt"
model = YOLOv10(model_path)

result = model(source = "/content/MRI-3/train/images", conf = 0.25, save=True)

<div style="background-color: #003366; color: #fff; padding: 10px; border-radius: 5px; text-align: center;">
    <h3>DISPALYING IMAGES</h3>
</div>

In [ ]:
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

images = glob.glob("/content/runs/detect/predict/*.jpg")

images_to_display = images[:10]

fig, axes = plt.subplots(2,5, figsize=(20,10))

for i, ax in enumerate(axes.flat):
  if i < len(images_to_display):
    img = mpimg.imread(images_to_display[i])
    ax.imshow(img)
    ax.axis('off')
  else:
    ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
result = model.predict(source = "/content/MRI-3/valid/images/Tr-gl_0228_jpg.rf.b9ecef834d39f770e41b0585b63bdc1a.jpg", imgsz = 640, conf = 0.25)
annotated_img = result[0].plot()
annotated_img[:, :, ::-1]

<div style="background-color: #003366; color: #fff; padding: 10px; border-radius: 5px; text-align: center;">
    <h3>MAKING PREDICTIONS</h3>
</div>

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
import cv2
import numpy as np

def predict(image):
  result = model.predict(source=image, imgsz = 640, conf = 0.25)
  annotated_img = result[0].plot()
  annotated_img = annotated_img[:, :, ::-1]
  return annotated_img

app = gr.Interface(
    fn =predict,
    inputs = gr.Image(type="numpy", label="Upload an image"),
    outputs = gr.Image(type="numpy", label="Detect Brain Tumor"),
    title = "Brain Tumor Detection Using Yolov10",
    description="Upload an image and the yolov10 model will detect and annotated brain tumor"
)

app.launch()